This notebook contains two queries that, given a Wikidata proprety as input, builds a list of the assigned classes, English labels, and outgoing properties for all subjects and objects that are causing a constraint violation of the given property. Label and assigned classes (P31) are searched as Optionals so that blank results are also returned.

In [1]:
import os
import csv
import numpy as np
import pandas as pd

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [11]:
input_path = "/out/data"
output_path = "output"
project_name = "p161_notebook"

# set property ID here
property_id = "P50"

files = [f"claims.{property_id}", f"labels.subjects.{property_id}", f"instances.subjects.{property_id}.tsv"]
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)
#ck.print_env_variables()
#ck.load_files_into_cache()

User home: /home/jovyan
Current dir: /kgtk/move_me
KGTK dir: /kgtk
Use-cases dir: /kgtk/use-cases


In [12]:
kgtk(f"""--debug query 
-i /out/output/allConstraintsAnalysis_Final/typeConstraint/normal/claims.type-constraints.instanceOf.{property_id}.incorrect.tsv --as typeClaims_{property_id}
-i /out/labels.en.tsv.gz --as labels2
-i /out/derived.P31.tsv.gz --as instances2
-i /out/claims.wikibase-item.tsv.gz --as wikibase
--match 'typeClaims_{property_id}: (node1)-[nodeProp]->(node2)'
--opt 'labels2: (node1)-[:label]->(lb)'
--opt 'instances2: (node1)-[:P31]->(inst)'
--opt 'wikibase: (node1)-[someProp]->(someNode)'
--return 'distinct node1 as `node1`, lb as `label`, inst as `instanceOf`, someNode as `otherConn`'
-o /out/output/allConstraintsAnalysis_Final/missing_instances/missing_labels.subjects.{property_id}.tsv 
""")

[2022-01-06 19:48:53 sqlstore]: IMPORT graph directly into table graph_16 from /out/output/allConstraintsAnalysis_Final/typeConstraint/normal/claims.type-constraints.instanceOf.P50.incorrect.tsv ...
[2022-01-06 19:48:53 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_16_c1."node1" "_aLias.node1", graph_9_c2."node2" "_aLias.label", graph_10_c3."node2" "_aLias.instanceOf", graph_11_c4."node2" "_aLias.otherConn"
     FROM graph_16 AS graph_16_c1
     LEFT JOIN graph_9 AS graph_9_c2
     ON graph_16_c1."node1" = graph_9_c2."node1"
        AND graph_9_c2."label" = ?
     LEFT JOIN graph_10 AS graph_10_c3
     ON graph_16_c1."node1" = graph_10_c3."node1"
        AND graph_10_c3."label" = ?
     LEFT JOIN graph_11 AS graph_11_c4
     ON graph_16_c1."node1" = graph_11_c4."node1"
  PARAS: ['label', 'P31']
---------------------------------------------
[2022-01-06 19:48:53 sqlstore]: CREATE INDEX "graph_16_node1_idx" ON "graph_16" ("node1")
[2022-01-

In [13]:
kgtk(f"""--debug query 
-i /out/output/allConstraintsAnalysis_Final/valueTypeConstraint/normal/claims.type-constraints.instanceOfOrSubclass.{property_id}.incorrect.tsv --as valueTypeClaims_{property_id}
-i /out/labels.en.tsv.gz --as labels2
-i /out/derived.P31.tsv.gz --as instances2
-i /out/claims.wikibase-item.tsv.gz --as wikibase
--match 'valueTypeClaims_{property_id}: (node1)-[nodeProp]->(node2)'
--opt 'labels2: (node2)-[:label]->(lb)'
--opt 'instances2: (node2)-[:P31]->(inst)'
--opt 'wikibase: (node2)-[someProp]->(someNode)'
--return 'distinct node2 as `node2`, lb as `label`, inst as `instanceOf`, someNode as `otherConn`'
-o /out/output/allConstraintsAnalysis_Final/missing_instances/missing_labels.objects.{property_id}.tsv 
""")

[2022-01-06 19:49:02 sqlstore]: IMPORT graph directly into table graph_17 from /out/output/allConstraintsAnalysis_Final/valueTypeConstraint/normal/claims.type-constraints.instanceOfOrSubclass.P50.incorrect.tsv ...
[2022-01-06 19:49:03 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_17_c1."node2" "_aLias.node2", graph_9_c2."node2" "_aLias.label", graph_10_c3."node2" "_aLias.instanceOf", graph_11_c4."node2" "_aLias.otherConn"
     FROM graph_17 AS graph_17_c1
     LEFT JOIN graph_9 AS graph_9_c2
     ON graph_17_c1."node2" = graph_9_c2."node1"
        AND graph_9_c2."label" = ?
     LEFT JOIN graph_10 AS graph_10_c3
     ON graph_17_c1."node2" = graph_10_c3."node1"
        AND graph_10_c3."label" = ?
     LEFT JOIN graph_11 AS graph_11_c4
     ON graph_17_c1."node2" = graph_11_c4."node1"
  PARAS: ['label', 'P31']
---------------------------------------------
[2022-01-06 19:49:03 sqlstore]: CREATE INDEX "graph_17_node2_idx" ON "graph_17" ("no